In [0]:
import pandas as pd
import random as r
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier
r.seed(42)


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [0]:
r.seed(42)
MINRECORDS=200
exos=pd.read_csv("exoplanets.csv")
imputedNumericCols=pd.read_csv("ImputedNumericCols.csv")
habitable=[2902, 3132, 2155, 2021, 1147, 2541, 2503, 1845, 2135, 2538, 3233, 2880, 2156, 163, 2097, 1424, 3115, 2883, 986, 2316, 1227, 2441, 2031, 2014, 3606, 2129, 703, 114, 2547, 1137, 128, 2189,151, 152, 153, 1604, 2155, 2223, 2882, 3133, 3606, 3716, 3742,3743, 3744]
for n in imputedNumericCols.columns:
	exos[n]=imputedNumericCols[n]
remove=['pl_hostname', 'pl_name', 'ra_str', 'dec_str', 'rowupdate', 'pl_def_reflink', 'pl_disc_reflink', 'pl_pelink', 'pl_edelink', 'pl_publ_date', 'hd_name', 'hip_name', 'st_spstr', 'swasp_id']
exos=exos.drop(remove,axis=1)
for n in exos.columns:
	if len(exos)-exos[n].isna().sum()<MINRECORDS:
		exos.drop(n,axis=1)
print(len(exos))
Cat=[x for x in exos.columns if x not in exos._get_numeric_data().columns]
exos[Cat]=exos[Cat].fillna(value="Missing")
for n in Cat:
	exos[n]=pd.Categorical(exos[n])
exos['habitable']=exos.rowid.isin(habitable).replace(True,1).rename('habitable')


# smote = SMOTE(ratio='minority')




3970


In [0]:
r.seed(42)
def enoughdata(df):
	trainCols=list(df.columns)
	trainCols.remove('rowid')
	trainCols.remove('habitable')
	trainCols.sort()
	trainCols=",".join(trainCols)
	if(trainCols in out['features']):
		return 0
	num=0
	for n in habitable:
		if n in df.rowid:
			num+=1
	if num<10:
		return 0
	if len(data)<MINRECORDS:
		return 0
	return trainCols

def Preprocessing(df):
	# df=df.set_index(keys="rowid",drop=False)
	nonCat=list(df._get_numeric_data().columns)
	nonCat.remove('rowid')
	nonCat.remove('habitable')
	cat=[]
	df[nonCat]=MinMaxScaler().fit_transform(df[nonCat])
	for n in df.columns:
		if n not in nonCat and n not in['rowid','habitable']:
			cat.append(n)
		
	df=pd.get_dummies(df,columns=cat)

	trainCols=[x for x in df.columns if x not in ['habitable','rowid']]
	validate=[]
	Hcopy=habitable.copy()
	for n in Hcopy:
		if n not in df.rowid:
			Hcopy.remove(n)
	#numHidden sets the number of habitable planets to use for validation of the model

	numHidden=round(len(Hcopy)/2)
	#Randomly select "numHidden" number of habitable exoplanets and
	#add their row_id to validation set 
	for i in range(numHidden): 
		randNum=r.randint(0,len(Hcopy)-1)	    
		validate.append(Hcopy[randNum])	    
		del Hcopy[randNum]	    
	#Add row_id of non-habitable planets to the validation set till its length becomes 100

	while len(validate)<50:	
		temp=r.randint(0,len(df))	    
		if temp not in Hcopy and temp in df.rowid:	    
			validate.append(temp)	        
	#Take all columns of the planets whose row_id is in "validate" variable(in the validation set) and
	#store it in "validate" variable
	validate=df[df.rowid.isin(validate)]
	#Store the planets that are not in the validation set in the training set
	trainingSet=df[~df.rowid.isin(validate.rowid)]

	#Store the training features in X and target feature(habitable or not) in y 
	X_sm=trainingSet[trainCols]
	y_sm=trainingSet.habitable

	#the SMOTE library mutates existing data to creating more data
	#Here we use SMOTE to increase the number of habitable planets in the training and validation data
	a=pd.concat([X_sm,y_sm])
	validateX=validate[trainCols]
	validateY=validate.habitable
	return X_sm,y_sm,validateX,validateY





In [0]:
out={'features':[],'score':[]}
MinLen=500
numTests=0
numCols=len(exos.columns)
r.seed(42)
q=0
while numTests<10000:
    q+=1
    features=list(exos.columns)
    features.remove('rowid')
    features.remove('habitable')
    SelectedFeats=['rowid','habitable']
    numFeats=r.randint(3,numCols-5)
	# print("NUMFEATS",len(features))
    for i in range(numFeats):
        featureNum=r.randint(0,len(features)-1)
        SelectedFeats.append(features[featureNum])
        del features[featureNum]
    if((",").join(SelectedFeats) in out['features']):
        continue
    data=exos[SelectedFeats].copy()
    data=data.dropna()
    trainCols=enoughdata(data)
    if trainCols:
        numTests+=1
        print(str(numTests)+")","Num features",numFeats)
        X_sm,y_sm,validateX,validateY=Preprocessing(data)
        model=KNeighborsClassifier(n_neighbors=5,algorithm='auto',leaf_size=10,p=1,n_jobs=-1)
        model.fit(X_sm,y_sm)
        out['features'].append(trainCols)
        out['score'].append(balanced_accuracy_score(validateY,model.predict(validateX)))
        if(numTests%100==0):
# Create & upload a text file.
            pd.DataFrame(out).to_csv('RandomFeatures.csv')
#     print("Fail",q,end=" ")    

1) Num features 4
2) Num features 6
3) Num features 5
4) Num features 5
5) Num features 3
6) Num features 3
7) Num features 3
8) Num features 4
9) Num features 3
10) Num features 3
11) Num features 3
12) Num features 4
13) Num features 6
14) Num features 5
15) Num features 3
16) Num features 3
17) Num features 3
18) Num features 3
19) Num features 3
20) Num features 3
21) Num features 3
22) Num features 4
23) Num features 5
24) Num features 6
25) Num features 3
26) Num features 3
27) Num features 4
28) Num features 4
29) Num features 5
30) Num features 4
31) Num features 3
32) Num features 3
33) Num features 3
34) Num features 4
35) Num features 3
36) Num features 4
37) Num features 7
38) Num features 4
39) Num features 6
40) Num features 3
41) Num features 4
42) Num features 3
43) Num features 5
44) Num features 5
45) Num features 3
46) Num features 6
47) Num features 4
48) Num features 7
49) Num features 5
50) Num features 4
51) Num features 5
52) Num features 4
53) Num features 6
54

ValueError: ignored